In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys
from tensorflow.contrib import rnn

In [6]:
tf.reset_default_graph()

#입력값 준비
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
# hidden_size 로 출력의 개수를 정해준다
#모델 준비 num_units 는 히든사이즈 4개 입력받아서 3개로출력하겠따.
cell = tf.contrib.rnn.BasicRNNCell(num_units = 3)

#입력 데이터 준비
# h 셀하나만 해본거
# x_data = np.array([[h]], dtype=np.float32)
#5개의 쉘
x_data = np.array([[h,e,l,l,o]], dtype=np.float32)
print(x_data)

#RNN 실행
outputs, _state=tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(outputs))
print("----------------------")
print(sess.run(_state))

[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]
[[[ 0.07634945 -0.28618345 -0.43884867]
  [ 0.05115039  0.34016976  0.69185114]
  [-0.63008577 -0.47533223  0.04679133]
  [-0.3451375   0.33163545  0.568166  ]
  [ 0.07249249  0.10690822 -0.31662282]]]
----------------------
[[ 0.07249249  0.10690822 -0.31662282]]


In [14]:
tf.reset_default_graph()
# 숫자로 입력의 개수는 2개 (1,2) 가 셀로 들어간다
# 전에는 4개가 들어갔으면 이번에는 2개
inputs = np.array([[[1,2]]])

tf.set_random_seed(777)
tf._inputs = tf.constant(inputs, dtype = tf.float32)
# 출력을 3개로 하겠다. 
cell = tf.nn.rnn_cell.BasicRNNCell(num_units = 3)
#tf_inputs을 넣는다. 
outputs, _state = tf.nn.dynamic_rnn(cell=cell, dtype = tf.float32, inputs = tf._inputs)
variabels_names = [v.name for v in tf.trainable_variables()]

print(outputs) #Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
print(_state)

print("weights")
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
    
print("-----------------------")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, _state])
    
    print("output values")
    print(output_run)
    
    print("\nstate values")
    print(state_run)
    
    print("\nweights")
    values = sess.run(variabels_names)
    for k, v in zip(variabels_names, values):
        print(k, v)

Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)
weights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
-----------------------
output values
[[[-0.9314169   0.75578666 -0.6819246 ]]]

state values
[[-0.9314169   0.75578666 -0.6819246 ]]

weights
rnn/basic_rnn_cell/kernel:0 [[-0.62831575  0.38538355  0.79733914]
 [-0.5203329   0.30046564 -0.8150209 ]
 [ 0.39399797  0.16670114  0.4062907 ]
 [-0.6391754   0.8460203   0.5266966 ]
 [ 0.41124135  0.66347724 -0.0210759 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


In [16]:
# I work at google = [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]]
# I google at work = [[1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0]]

# 2개의 문장을 입력 데이터로 
tf.reset_default_graph()

inputs = np.array([
    [[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]],
    [[1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0]]
])
tf._inputs = tf.constant(inputs, dtype = tf.float32)
# 출력을 3개로 하겠다. 
cell = tf.nn.rnn_cell.BasicRNNCell(num_units = 3)

outputs, _state = tf.nn.dynamic_rnn(cell=cell, dtype = tf.float32, inputs = tf._inputs)
variabels_names = [v.name for v in tf.trainable_variables()]

print(outputs)
print(_state)

print("weights")
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)

Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
weights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-12-1-hello-rnn.py

In [43]:
# hihello 학습

tf.reset_default_graph()
tf.set_random_seed(777)

# 파라미터 
num_classes=5
input_dim = 5
hidden_size = 5
batch_size = 1
sequence_len = 7
lr = 0.1

idx2char = ['h','i','e','l','o']
x_one_hot = [[
    [1,0,0,0,0], #h
    [0,1,0,0,0], #i
    [1,0,0,0,0],#h
    [0,0,1,0,0],#e
    [0,0,0,1,0],#l
    [0,0,0,1,0],#l
    [0,0,0,0,1],#o
]]

x_data = [[0,1,0,2,3,3]] # hihell
y_data = [[0,1,0,2,3,3,4]] # hihello

# X, y 변수 입력데이터의 shape을 어떻게할까
# 맨마지막이 각각의 입력데이터 갯수 원핫이 5개, 2번쨰인자 : 쉘의 개수
# 2번째인자 : 쉘 sequence는 6개?? , 1번째 인자 : batch_size 입력할필요없음
X = tf.placeholder(tf.float32, shape = [None,sequence_len,input_dim])
y = tf.placeholder(tf.int32, [None,sequence_len])

cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
outputs, _state = tf.nn.dynamic_rnn(cell=cell,inputs =X,dtype=tf.float32,
                                    initial_state=cell.zero_state(1,tf.float32))                                    
print(outputs)
# 평면화 -1 : 행의 개수 상관없이 5개로
X_for_fc=tf.reshape(outputs, [-1, hidden_size])

# FC Layer
# W, b, logit 입력의 개수 출력의 개수 활성화 함수 뭐쓸건지 해주는 함수
outputs = tf.contrib.layers.fully_connected(inputs = X_for_fc,
                                  num_outputs = 5, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size,sequence_len,num_classes])
weights = tf.ones([batch_size,sequence_len])
seq_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs ,
                                            targets=y, weights=weights)

loss = tf.reduce_mean(seq_loss)
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

Tensor("rnn/transpose_1:0", shape=(1, 7, 5), dtype=float32)


In [23]:
x_one_hot

[[[1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [1, 0, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1]]]

In [44]:
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        
        if i%100 ==0:
            print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.6383449 prediction:  [[0 3 3 3 3 3 2]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hllllle
	Prediction str:  hhhlllo
	Prediction str:  hhhlllo
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Predict

	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction 

	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction 

	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction 

	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
	Prediction str:  hihello
1700 loss: 2.843943e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  

# LSTM

In [46]:
tf.reset_default_graph()
tf.set_random_seed(777)

inputs = np.array([[[1,0]]])
tf_inputs = tf.constant(inputs, dtype = tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(num_units = 1) # hiddensize
outputs, _state = tf.nn.dynamic_rnn(cell= cell, dtype = tf.float32,
                                   inputs = tf_inputs)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run, state_run = sess.run([outputs, _state])
    print("output values")
    print(output_run)
    print("\nhidden values")
    print(state_run.h)
    print("\nmemory cell")
    print(state_run.c)

output values
[[[-0.9314169   0.75578666 -0.6819246 ]]]

hidden values
[[0.09927537]]

memory cell
[[0.18134572]]


### 사례 : LSTM 을 이용한 문장 분류 

In [47]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

In [48]:
df = pd.DataFrame(paragraph_dict_list)
df = df[["paragraph", "category"]]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


In [49]:
results = set()
df["paragraph"].str.lower().str.split().apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [50]:
results

{'(and',
 '1',
 '1.5',
 '1.5x',
 '10-9',
 '14',
 '2',
 '2014',
 '33/1',
 'a',
 'about',
 'action',
 'after',
 'against',
 'ago',
 'ai',
 'alisson',
 'all',
 'almost',
 'also',
 'although',
 'alto',
 'amazing',
 'american',
 'and',
 'angelique',
 'any',
 'apps',
 'are',
 'area',
 'around',
 'as',
 'at',
 'australia',
 'avoided',
 'azarenka',
 'back',
 'battle',
 'be',
 'beef',
 'been',
 'before',
 'began',
 'began—too',
 'being',
 'better',
 'between',
 'big',
 'biscuits',
 'bit',
 'bookmark',
 'bookmarked',
 'both',
 'bottom',
 'brazilian',
 'bread',
 'breakthroughs',
 'briton',
 'brunch',
 'burger',
 'burgers',
 'business',
 'busy',
 'but',
 'by',
 'came',
 'campaigns',
 'can',
 'card',
 'caroline',
 'case',
 'certainly',
 'champion',
 'champions',
 'chance',
 'chances',
 'chicken',
 'city',
 'clay',
 'clean',
 'commented',
 'competition',
 'complain',
 'confidence',
 'contrast',
 'could',
 'course',
 'courteous',
 'courts',
 'coworkers',
 'crisp',
 'croatia',
 'crumbs',
 'culminating

In [51]:
idx2word = dict(enumerate(results))
idx2word

{0: 'hour',
 1: 'themselves',
 2: 'serves',
 3: 'garlic',
 4: 'battle',
 5: 'time',
 6: 'open',
 7: 'began—too',
 8: 'restaurants',
 9: 'left',
 10: 'generous',
 11: 'big',
 12: 'area',
 13: 'title',
 14: 'results',
 15: 'serving',
 16: 'quartet',
 17: 'ranked',
 18: 'important',
 19: 'than',
 20: 'england',
 21: 'rakitic',
 22: 'italico',
 23: 'card',
 24: 'downtown',
 25: 'no1',
 26: 'summer',
 27: 'winning',
 28: 'excellent',
 29: 'quarter-final',
 30: 'slides',
 31: 'semis',
 32: 'enormously',
 33: 'half',
 34: 'full',
 35: 'said',
 36: 'never',
 37: 'miami',
 38: 'third',
 39: 'succeed',
 40: 'eliminated',
 41: 'djokovic-nadal',
 42: 'make',
 43: 'knock',
 44: 'premier',
 45: 'to',
 46: 'more',
 47: 'any',
 48: 'this',
 49: 'peak',
 50: 'jonesing',
 51: 'hubby',
 52: '1.5x',
 53: 'thought',
 54: 'rankings',
 55: 'there',
 56: 'food',
 57: 'kerber',
 58: 'formaggi',
 59: 'serena',
 60: 'week',
 61: 'edged',
 62: 'us',
 63: '14',
 64: 'fourth',
 65: 'fans',
 66: 'from',
 67: 'ago',


In [52]:
idx2word[396]
idx2word[100]

'just'

In [53]:
word2idx = {v:k for k, v in idx2word.items()}
word2idx

{'hour': 0,
 'themselves': 1,
 'serves': 2,
 'garlic': 3,
 'battle': 4,
 'time': 5,
 'open': 6,
 'began—too': 7,
 'restaurants': 8,
 'left': 9,
 'generous': 10,
 'big': 11,
 'area': 12,
 'title': 13,
 'results': 14,
 'serving': 15,
 'quartet': 16,
 'ranked': 17,
 'important': 18,
 'than': 19,
 'england': 20,
 'rakitic': 21,
 'italico': 22,
 'card': 23,
 'downtown': 24,
 'no1': 25,
 'summer': 26,
 'winning': 27,
 'excellent': 28,
 'quarter-final': 29,
 'slides': 30,
 'semis': 31,
 'enormously': 32,
 'half': 33,
 'full': 34,
 'said': 35,
 'never': 36,
 'miami': 37,
 'third': 38,
 'succeed': 39,
 'eliminated': 40,
 'djokovic-nadal': 41,
 'make': 42,
 'knock': 43,
 'premier': 44,
 'to': 45,
 'more': 46,
 'any': 47,
 'this': 48,
 'peak': 49,
 'jonesing': 50,
 'hubby': 51,
 '1.5x': 52,
 'thought': 53,
 'rankings': 54,
 'there': 55,
 'food': 56,
 'kerber': 57,
 'formaggi': 58,
 'serena': 59,
 'week': 60,
 'edged': 61,
 'us': 62,
 '14': 63,
 'fourth': 64,
 'fans': 65,
 'from': 66,
 'ago': 67,


In [55]:
# 원핫 인코딩 해주는 함수
def encode_paragraph(paragraph):
    words = paragraph.split(' ')
    encoded = []
    
    for word in words:
        encoded.append([word2idx[word]])
        
    return encoded

def encode_category(category):
    if category == 'food':
        return [1, 0]
    else:
        return [0, 1]
    
def word_cnt(paragraph):
    return len(paragraph.split(' '))

In [73]:
df["enc_paragraph"] = df.paragraph.apply(encode_paragraph)
df["enc_category"] = df.category.apply(encode_category)
df["seq_length"] = df.paragraph.apply(word_cnt)
df

,paragraph,category,enc_paragraph,enc_category,seq_lengh,seq_length
0,dishplace is located in sunnyvale downtown the...,food,"[[305], [215], [384], [462], [358], [24], [55]...","[1, 0]",53,53
1,service can be slower during busy hours but ou...,food,"[[338], [524], [143], [392], [468], [283], [32...","[1, 0]",19,19
2,portions are huge both french toast and their ...,food,"[[87], [273], [463], [357], [506], [68], [475]...","[1, 0]",42,42
3,we started with apps going the chicken and waf...,food,"[[411], [515], [482], [302], [86], [149], [346...","[1, 0]",43,43
4,the biscuits and gravy was too salty two peopl...,food,"[[149], [420], [475], [491], [324], [278], [51...","[1, 0]",82,82
5,the garlic fries were a great starter (and a h...,food,"[[149], [3], [151], [205], [73], [157], [486],...","[1, 0]",24,24
6,our meal was excellent i had the pasta ai form...,food,"[[267], [199], [324], [28], [131], [96], [149]...","[1, 0]",50,50
7,what i enjoy most about palo alto is so many r...,food,"[[494], [131], [457], [70], [309], [207], [296...","[1, 0]",43,43
8,the drinks came out fairly quickly a good two ...,food,"[[149], [251], [354], [431], [391], [102], [73...","[1, 0]",49,49
9,despite the not so good burger the service was...,food,"[[112], [149], [375], [438], [204], [257], [14...","[1, 0]",82,82


In [57]:
max_word_cnt = 0
for row in df["paragraph"]:
    if len(row.split(" ")) > max_word_cnt:
        max_word_cnt = len(row.split(" "))
        
print(max_word_cnt)

91


In [58]:
# 문장의 길이가 91개가 안될 경우 -1로 패딩 처리
def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1])
        
    return enc_paragraph

In [59]:
df["enc_paragraph"] = df.enc_paragraph.apply(sequence_padding)

In [61]:
print(df["enc_paragraph"][0])

[[305], [215], [384], [462], [358], [24], [55], [215], [387], [443], [149], [12], [104], [158], [524], [143], [219], [45], [200], [468], [49], [323], [321], [245], [217], [475], [131], [354], [45], [48], [485], [416], [317], [259], [73], [206], [334], [205], [139], [283], [438], [131], [369], [99], [189], [237], [308], [276], [289], [149], [193], [45], [424], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]


In [70]:
# 입력값들을 비열로 변환
enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.enc_category.tolist())
seq_length = np.array(df.seq_lengh.tolist())

In [71]:
train_X = enc_paragraph
train_y = enc_category

print(train_X.shape)
print(train_y.shape)

(20,)
(20, 2)


In [72]:
# 모델 구축
tf.reset_default_graph()
tf.set_random_seed(777)

lr = 0.001
n_epochs = 300

X = tf.placeholder(tf.float32, [None, max_word_cnt, ])
y = tf.placeholder(tf.int32, [None, 2])

cell = tf.nn.rnn_cell.LSTMCell(num_units = 64)
outputs, _state = tf.nn.dynamic_rnn(cell = cell, inputs = X, dtype = tf.float32,
                                   sequence_length=seq_length)
# W3, b1 , logit 활성화 함수까지 합친 히든 계층
dense_layer = tf.layers.dense(_state.h, 32)
logits = tf.layers.dense(dense_layer, 2)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y,
                                                          logits = logits)
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

ValueError: Shape (91, ?) must have rank at least 3

In [ ]:
print(X)
print(_state)
print(dense_layer)
print(logits)